In [62]:
import os
import json
import pytz
import pandas as pd
import numpy as np
from pathlib import Path
from ast import literal_eval
# import Source.db_functions as db

from dateutil.tz import tz
from sqlalchemy import engine

## Main functions to manipulate a dataframe

### For testing purposes of data manipulation functions

In [63]:
def get_files_in_directory() -> list:
    """
    Sets a path to JSON file.
    :returns a file name in a set path
    """
    path_to_files = '../data/input/'
    files_in_path = os.scandir(path_to_files)
    
    list_of_files = []
    for file in files_in_path:
        if file.is_dir() or file.is_file():
            list_of_files.append(file.name)
            return list_of_files    # <- don't forget to align this statement back with for!!!!!

In [64]:
def create_dataframe(file_json: str) -> pd.DataFrame:
    """
    Creates a pandas dataframe from JSON file.
    Sets the maximum available columns to be shown.
    Requires name of the file.
    """
    path_to_files = '../data/input/'
    with open(path_to_files + file_json) as jfile:
        json_data = json.load(jfile)
        df = pd.DataFrame(pd.json_normalize(json_data))
        pd.set_option('display.max_columns', None)
    return df

In [65]:
# FLATTEN JSON FILE!!!
def flatten_json_file(dataframe: pd.DataFrame, row: str) -> pd.DataFrame:
    """
    Flattens the supplied dataframe and returns a new dataframe with flattened json data.
    :param dataframe: dataframe to flatten
    :return: new dataframe with flattened json data
    """
    dataframe_flat = dataframe[row].apply(pd.Series)
    dataframe_flat_0 = dataframe_flat[0].apply(pd.Series)
    dataframe_flat_0.columns = ['weather_id', 'weather_main', 'weather_description', 'weather_icon']
    dataframe_new = pd.concat([dataframe, dataframe_flat_0], axis=1)
    dataframe_new = dataframe_new.drop(columns=['weather'], axis=1)
    return dataframe_new

In [99]:
def change_column_names(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Changes the column names of the dataframe to their new column names.
    :param dataframe: a pandas dataframe to change column names
    :return: dataframe with new column names
    """
    new_names = {"dt": "date_local",
                 "name": "city",
                 "id": "country_id", 
                 "coord.lon": "longitude",
                 "coord.lat": "latitude",
                 "main.temp": "main_temp",
                 "main.feels_like": "main_feels_like",
                 "main.temp_min": "main_temp_min",
                 "main.temp_max": "main_temp_max",
                 "main.pressure": "pressure",
                 "main.humidity": "humidity",
                 "wind.speed": "wind_speed",
                 "wind.deg": "wind_deg",
                 "clouds.all": "clouds",
                 "sys.type": "sys_type",
                 "sys.id": "sys_id",
                 "sys.country": "country",
                 "sys.sunrise": "sunrise_local",
                 "sys.sunset": "sunset_local"}
    dataframe.rename(columns=new_names, inplace=True)
    return dataframe

In [214]:
def change_datetime_format(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Change datetime format of a given dataframe to ISO 8601 format
    :param dataframe: dataframe to change datetime format for.
    :return: dataframe with changed datetime format
    """
    date_cols = ['date_local', 'sunrise_local', 'sunset_local']
    local_timezone = pytz.timezone('Europe/Vilnius')
    dataframe['date_vilnius'] = (pd.to_datetime(dataframe['date_local'], unit='s', errors='coerce', utc=True)
                                 .dt.tz_convert(local_timezone))

    for i in date_cols:
        dataframe[i] = dataframe[i] + dataframe['timezone']
        dataframe[i] = pd.to_datetime(dataframe[i], unit='s', errors='coerce')
        
    return dataframe

In [92]:
def reorder_dataframe_columns(dataframe: pd.DataFrame) -> pd.DataFrame:
    reordered_columns = ['longitude', 'latitude', 'country_id', 'country', 'city', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max', 'date_vilnius', 'date_local', 'timezone', 'sunrise_local', 'sunset_local', 'weather_id', 'weather_main', 'weather_description', 'weather_icon', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds', 'visibility', 'base', 'sys_type', 'sys_id', 'cod']
    
    dataframe = dataframe[reordered_columns]
    return dataframe

In [69]:
def inspect_dataframe(dataframe: pd.DataFrame):
    """
    Returns information about the dataset 
    and shows a preview of the data in it.
    """
    print(dataframe.info(), '\n')
    print(dataframe.head())

In [9]:
def load_to_database(dataframe: pd.DataFrame, table_name: str) -> None:
    """
    Function to load the data of a dataframe to a specified table in the database.
    :param dataframe: dataframe to load data from.
    :param table_name: table to load the data to.
    :return: None
    """
    
    dataframe.to_sql(table_name, engine, if_exists='append')

## Finding the longest value in each column of a given dataframe

In [70]:
def longest_value(dataframe: pd.DataFrame):
    """
    Checking for the longest value in each column of a given dataframe.
    """
    for row in dataframe:
        col_name = row

        if dataframe[row].dtype == float or int:
            list_len = dataframe[row].astype(str).str.len().max()

        else:
            list_len = max(list(map(len, dataframe[row].values)))

        print('Column:', col_name)
        print('Value length:', list_len, '\n')

## Files in ../data/input location

In [215]:
cities = get_files_in_directory()
print(cities)

['Barcelona, Spain_response.json']


In [216]:
for city in cities:
    city_df = create_dataframe(city)

city_df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,rain.1h,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,10000,1708103350,3600,3128760,Barcelona,200,2.159,41.3888,14.13,13.58,13.08,15.72,1021,76,3.6,140,0.11,75,2,18549,ES,1708065970,1708104303


In [217]:
flatten_df = flatten_json_file(city_df, 'weather')
flatten_df.head()

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,rain.1h,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,weather_id,weather_main,weather_description,weather_icon
0,stations,10000,1708103350,3600,3128760,Barcelona,200,2.159,41.3888,14.13,13.58,13.08,15.72,1021,76,3.6,140,0.11,75,2,18549,ES,1708065970,1708104303,500,Rain,light rain,10d


In [218]:
names_df = change_column_names(flatten_df)
names_df.head()

,base,visibility,date_local,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,rain.1h,clouds,sys_type,sys_id,country,sunrise_local,sunset_local,weather_id,weather_main,weather_description,weather_icon
0,stations,10000,1708103350,3600,3128760,Barcelona,200,2.159,41.3888,14.13,13.58,13.08,15.72,1021,76,3.6,140,0.11,75,2,18549,ES,1708065970,1708104303,500,Rain,light rain,10d


In [119]:
names_df['date_local_new'] = names_df['date_local'] + names_df['timezone']
print(names_df['date_local_new'])
print(names_df['date_local_new'])
names_df['date_local_new_2'] = pd.to_datetime(names_df['date_local_new'], unit='s', errors='coerce', utc=True)
print(names_df['date_local_new_2'])
names_df['date_vilnius'] = names_df['date_local_new_2'].dt.tz_convert('Europe/Vilnius')
print(names_df['date_vilnius'])

0    1708102651
Name: date_local_new, dtype: int64
0    1708102651
Name: date_local_new, dtype: int64
0   2024-02-16 16:57:31+00:00
Name: date_local_new_2, dtype: datetime64[ns, UTC]
0   2024-02-16 18:57:31+02:00
Name: date_vilnius, dtype: datetime64[ns, Europe/Vilnius]


In [58]:
# normalize date to my local timezone
import pytz

local_timezone = pytz.timezone('Europe/Vilnius')
names_df['date'] = pd.to_datetime(names_df['date'], unit='s', errors='coerce', utc=True)
names_df['date_df'] = names_df['date'].dt.tz_convert(local_timezone)

# names_df['date_local'] = names_df['date_df'].dt.tz_convert(local_timezone)
print(local_timezone)
names_df.head()

Europe/Vilnius


,base,visibility,date,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sys_type,sys_id,country,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon,date_df
0,stations,10000,2024-02-15 15:59:01+00:00,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,1707979649,1708017828,800,Clear,clear sky,01d,2024-02-15 17:59:01+02:00


In [219]:
dates_df = change_datetime_format(names_df)
dates_df.head()

,base,visibility,date_local,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,rain.1h,clouds,sys_type,sys_id,country,sunrise_local,sunset_local,weather_id,weather_main,weather_description,weather_icon,date_vilnius
0,stations,10000,2024-02-16 18:09:10,3600,3128760,Barcelona,200,2.159,41.3888,14.13,13.58,13.08,15.72,1021,76,3.6,140,0.11,75,2,18549,ES,2024-02-16 07:46:10,2024-02-16 18:25:03,500,Rain,light rain,10d,2024-02-16 19:09:10+02:00


In [220]:
reordered_df = reorder_dataframe_columns(dates_df)
reordered_df.head()

,longitude,latitude,country_id,country,city,main_temp,main_feels_like,main_temp_min,main_temp_max,date_vilnius,date_local,timezone,sunrise_local,sunset_local,weather_id,weather_main,weather_description,weather_icon,pressure,humidity,wind_speed,wind_deg,clouds,visibility,base,sys_type,sys_id,cod
0,2.159,41.3888,3128760,ES,Barcelona,14.13,13.58,13.08,15.72,2024-02-16 19:09:10+02:00,2024-02-16 18:09:10,3600,2024-02-16 07:46:10,2024-02-16 18:25:03,500,Rain,light rain,10d,1021,76,3.6,140,75,10000,stations,2,18549,200


In [86]:
columns = dates_df.columns.tolist()

index_col = []
for i, col in enumerate(columns):
    index_col.append([i, col])
    
print(index_col)

[[0, 'base'], [1, 'visibility'], [2, 'date'], [3, 'timezone'], [4, 'country_id'], [5, 'city'], [6, 'cod'], [7, 'longitude'], [8, 'latitude'], [9, 'main_temp'], [10, 'main_feels_like'], [11, 'main_temp_min'], [12, 'main_temp_max'], [13, 'pressure'], [14, 'humidity'], [15, 'wind_speed'], [16, 'wind_deg'], [17, 'clouds'], [18, 'sys_type'], [19, 'sys_id'], [20, 'country'], [21, 'sunrise'], [22, 'sunset'], [23, 'weather_id'], [24, 'weather_main'], [25, 'weather_description'], [26, 'weather_icon']]


In [212]:
reordered_columns = ['longitude', 'latitude', 'country_id', 'country', 'city', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max', 'date_vilnius', 'date_local', 'timezone', 'sunrise_local', 'sunset_local', 'weather_id', 'weather_main', 'weather_description', 'weather_icon', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds', 'visibility', 'base', 'sys_type', 'sys_id', 'cod']

reordered_df = dates_df[reordered_columns]

reordered_df.head()

,longitude,latitude,country_id,country,city,main_temp,main_feels_like,main_temp_min,main_temp_max,date_vilnius,date_local,timezone,sunrise_local,sunset_local,weather_id,weather_main,weather_description,weather_icon,pressure,humidity,wind_speed,wind_deg,clouds,visibility,base,sys_type,sys_id,cod
0,2.159,41.3888,3128760,ES,Barcelona,14.17,13.63,13.08,16.22,2024-02-16 18:59:05+02:00,2024-02-16 17:59:05,3600,2024-02-16 07:46:10,2024-02-16 18:25:03,300,Drizzle,light intensity drizzle,09d,1021,76,3.6,140,75,10000,stations,2,18549,200


In [221]:
city_df_val_len = longest_value(reordered_df)

Column: longitude
Value length: 5 

Column: latitude
Value length: 7 

Column: country_id
Value length: 7 

Column: country
Value length: 2 

Column: city
Value length: 9 

Column: main_temp
Value length: 5 

Column: main_feels_like
Value length: 5 

Column: main_temp_min
Value length: 5 

Column: main_temp_max
Value length: 5 

Column: date_vilnius
Value length: 25 

Column: date_local
Value length: 19 

Column: timezone
Value length: 4 

Column: sunrise_local
Value length: 19 

Column: sunset_local
Value length: 19 

Column: weather_id
Value length: 3 

Column: weather_main
Value length: 4 

Column: weather_description
Value length: 10 

Column: weather_icon
Value length: 3 

Column: pressure
Value length: 4 

Column: humidity
Value length: 2 

Column: wind_speed
Value length: 3 

Column: wind_deg
Value length: 3 

Column: clouds
Value length: 2 

Column: visibility
Value length: 5 

Column: base
Value length: 8 

Column: sys_type
Value length: 1 

Column: sys_id
Value length: 5 

Co

In [142]:
city_inspect = inspect_dataframe(reordered_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   longitude            1 non-null      float64            
 1   latitude             1 non-null      float64            
 2   country_id           1 non-null      int64              
 3   country              1 non-null      object             
 4   city                 1 non-null      object             
 5   main_temp            1 non-null      float64            
 6   main_feels_like      1 non-null      float64            
 7   main_temp_min        1 non-null      float64            
 8   main_temp_max        1 non-null      float64            
 9   date_vilnius         1 non-null      datetime64[ns]     
 10  date_local           1 non-null      datetime64[ns, UTC]
 11  timezone             1 non-null      int64              
 12  sunrise_local        1 non